Stress Microarray Grapher:
    This program allows you to filter the three microarrays comparing PACAP Knockout, Pac1 Knockout, and Control mice following restraint stress. First, set your threshold for p-value and fold-change. Pressing the "Set Threholds" button will make the gene selection pane appear. The genes available in the drop-down are only those that reached your previously established significance level in at least one comparison. Choose each gene in the dropdown and click "Enter Gene". WHen you have selected your genes, click "Produce Output." 
    
    
    NOTE- Reset is not working very well now- if you want to 

In [1]:
import matplotlib
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import ipywidgets as widgets
import io
import plotly.express as px
from IPython.display import display
from ipywidgets import interact_manual, interactive,GridspecLayout, AppLayout, Button, Layout, Widget

In [2]:
##DATA FOR STRESS COMPARISONS
#retreive file with ANOVA data
result = pd.read_csv(r"C:\Users\bakalarda\Desktop\Bioinformatics Rapgef2\EidenLab Microarray\for stress v nostress\Data\Processed Expression Matrixes\statistical_results.csv")
#load the expression data and combine/ fix older ones
a=pd.read_csv(r"C:\Users\bakalarda\Desktop\Bioinformatics Rapgef2\EidenLab Microarray\for stress v nostress\Data\Processed Expression Matrixes\expressions_2019.04 3hr restraint hypothalamus part1.csv")
b=pd.read_csv(r"C:\Users\bakalarda\Desktop\Bioinformatics Rapgef2\EidenLab Microarray\for stress v nostress\Data\Processed Expression Matrixes\expressions_2019.04 3hr restraint hypothalamus part2.csv")
c=pd.read_csv(r"C:\Users\bakalarda\Desktop\Bioinformatics Rapgef2\EidenLab Microarray\for stress v nostress\Data\Processed Expression Matrixes\expressions_2019.04 3hr restraint adrenal part1.csv")
d=pd.read_csv(r"C:\Users\bakalarda\Desktop\Bioinformatics Rapgef2\EidenLab Microarray\for stress v nostress\Data\Processed Expression Matrixes\expressions_2019.04 3hr restraint adrenal part2.csv")
e=pd.read_csv(r"C:\Users\bakalarda\Desktop\Bioinformatics Rapgef2\EidenLab Microarray\for stress v nostress\Data\Processed Expression Matrixes\expressions_2019.03 1hr restraint hypothalamus part1.csv")
f=pd.read_csv(r"C:\Users\bakalarda\Desktop\Bioinformatics Rapgef2\EidenLab Microarray\for stress v nostress\Data\Processed Expression Matrixes\expressions_2019.03 1hr restraint hypothalamus part2.csv")
frames =[a,b,c,d,e,f]
expressions = pd.concat(frames, sort=False).reset_index()
expressions.drop(expressions.columns[0], axis=1, inplace=True)
expressions.drop(expressions.columns[0], axis=1, inplace=True)

In [3]:
##define GUI 
     
def choose_gene(change):
    if dropdown.value not in genes2:
        if dropdown.value != '':
            genes2.append(dropdown.value)
            with outgenes:
                display(dropdown.value)
                
graph_out1 = widgets.Output()  
graph_out2 = widgets.Output()
app_out = widgets.Output()


def on_value_change(change):
    #just be here
    print('')

    
            
def on_reset_button_clicked(reset_button):
    app_out.clear_output()
    graph_out1.clear_output() 
    graph_out2.clear_output()
    outerrors.clear_output()
    outgenes.clear_output()
    outgraph.clear_output()

    
def on_go_button_clicked(go_button):
    mouse_means2, stats, sig_genes = get_vals(go_button)
#Mark errors
    if len(genes2) < 1:
        with outerrors:
            print("Choose at least one Gene")
    else:

       ###Graph expression   
        import plotly.graph_objects as go
        with graph_out1:

            fig = px.bar(mouse_means2, x="combo", y="average",
                        facet_row = "Gene Symbol",
                        color = "combo",
                        facet_col = "Experiment",
                        hover_data = ["Gene Symbol","average","Tissue"],
                        color_discrete_sequence= px.colors.sequential.Plasma,
                        error_y="error",

                  ).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))

            for a in fig.layout.annotations:
                        a.text = a.text.split("=")[1]
                        fig.update_yaxes(title_text='')
                        fig.update_xaxes(title_text='')
                        fig.update_layout(
                            title="Expression levels of selected genes in groups where threshold qualifications were met",
                            #xaxis_title="Group",
                            yaxis_title="Fold-change",
                        xaxis={'categoryorder':'total descending'})

            fig.show()

            ################################################################
        with graph_out2:
            fig = px.bar(stats, x="Comparison", y="Fold-change",
                       # width=800, height=500,
                        facet_col = "Gene Symbol",
                        color_discrete_sequence= px.colors.sequential.Plasma,
                        color = "p-value", range_color=[0,p_threshold.value], #Change so user input tells it what p and fc to use
                        hover_name = "Experiment",
                        hover_data = ["Tissue", "Fold-change", "p-value"]
                    )
                        #.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
            fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
            fig.update_layout(title="Fold-changes and p-values",
                             )

            fig.show()

def on_thresh_button_clicked(th_go_button):
    
    #first clear any existing stuff out
    
    graph_out1.clear_output() 
    graph_out2.clear_output()
    outerrors.clear_output()
    outgenes.clear_output()
    outgraph.clear_output()
    genes2=[]
    results_stringent = result[result["p-value"] <= p_threshold.value]#Change so user input tells it what p and fc to use
    results_stringent2=results_stringent[(results_stringent['Fold-change']>= fc_threshold.value) | (results_stringent['Fold-change']<= -fc_threshold.value)]
    sig_genes0 = results_stringent2["Gene Symbol"].tolist()
    sig_genes = list(set(sig_genes0))
    mouse_means1 = expressions[expressions["Gene Symbol"].isin(genes2)]
    mouse_means1['combo']=mouse_means1['Genotype'] + " " + mouse_means1['Stress Condition']
    mouse_means2 = (mouse_means1.groupby(['Experiment','Gene Symbol','Tissue','combo'])['Expression']
         .agg([('average','mean'),('error','sem')])
         .reset_index())
    stats = results_stringent2[results_stringent2["Gene Symbol"].isin(genes2)]

    #create next part of app
    dropdown.options = sig_genes
    with app_out:
        display(app)
    return(mouse_means2, stats)


def get_vals(go_button):
    results_stringent = result[result["p-value"] <= p_threshold.value]#Change so user input tells it what p and fc to use
    results_stringent2=results_stringent[(results_stringent['Fold-change']>= fc_threshold.value) | (results_stringent['Fold-change']<= fc_threshold.value)]
    sig_genes = results_stringent2["Gene Symbol"].tolist()
    mouse_means1 = expressions[expressions["Gene Symbol"].isin(genes2)]
    mouse_means1['combo']=mouse_means1['Genotype'] + " " + mouse_means1['Stress Condition']
    mouse_means2 = (mouse_means1.groupby(['Experiment','Gene Symbol','Tissue','combo'])['Expression']
         .agg([('average','mean'),('error','sem')])
         .reset_index())
    stats = results_stringent2[results_stringent2["Gene Symbol"].isin(genes2)]

    return(mouse_means2, stats, sig_genes)

In [5]:



########################################################################################################
#App Prep
style = {'description_width': 'initial'}
genes2 = []       
spacer = widgets.Label('')
genetitle = widgets.Label('Select Genes')

dropdown =  widgets.Combobox(
    options= ["sig_genes"],
    description='Gene',
    placeholder='Type here',
    layout={'width': '80%'})

   
outgenes = widgets.Output()   
outgraph = widgets.Output(layout={'border': '1px solid black'})  

        
gene_button = widgets.Button(description = 'Enter Gene', 
                button_style = '', 
                display='flex',
                flex_flow='column',
                align_items='stretch')
gene_button.on_click(choose_gene)

go_button = widgets.Button(description = 'Produce Output', 
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch')
go_button.on_click(on_go_button_clicked)

reset_button = widgets.Button(description = 'Reset App', 
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch')
reset_button.on_click(on_reset_button_clicked)


gene_button.observe(dropdown.value)
go_button.observe(dropdown.value)

p_threshold = widgets.BoundedFloatText(
    value=0.05,
    description='p-value:',
    disabled=False,
    width = '80%',
    min=0,
    max=0.1,
    step=0.01,
    style=style)
p_threshold.observe(on_value_change, names='value')

fc_threshold = widgets.BoundedFloatText(
    value=2,
    description='Fold-change (+/-):',
    disabled=False,
    min=1.5,
    max=20,
    step=0.5,
    width = '80%',
    style=style)
fc_threshold.observe(on_value_change, names='value')


graph_out1 = widgets.Output()  
graph_out2 = widgets.Output() 

##########################################################################
##Deploy the app

threshold_title = widgets.Label('First, set p and fold-change')
th_go_button = widgets.Button(description = 'Set Thresholds', 
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch',
                                   value = "no")
th_go_button.on_click(on_thresh_button_clicked)
genes_out_title = widgets.Label('Genes Chosen:')


genesbox = widgets.VBox([genetitle,dropdown])
genes_out = widgets.VBox([spacer,genes_out_title, outgenes])
buttons = widgets.HBox([gene_button, go_button, reset_button])
#cellsbox = widgets.VBox([celllabel, Cells_drop])
genechoices = widgets.HBox([genesbox, genes_out])

outerrors = widgets.Output() 

graphit = widgets.VBox([graph_out1, graph_out2])
top_app = widgets.VBox([genechoices, outerrors, buttons]) ##Put seletcors for p and fc values here
app = widgets.VBox([top_app, graphit])
thresholds = widgets.VBox([threshold_title,p_threshold, fc_threshold, th_go_button,app_out])

thresholds